In [188]:
import copy
import multiprocessing as mp
import multiprocessing.dummy as dummy_mp
import os
import re
from datetime import datetime
from string import Template

import pymorphy2
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

FB_LOGIN = '89026051080'
FB_PASSWORD = 'z6A-8Wd-XiV-VNY'
login, password = FB_LOGIN, FB_PASSWORD

__base_url = 'https://www.facebook.com'
__base_group_url = 'https://www.facebook.com/groups'
__login_url = 'https://www.facebook.com/login'

options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-gpu")

options.add_argument("--disable-software-rasterizer")  # maybe not needed
options.add_argument("--disable-dev-shm-usage")  # maybe not needed

browser = webdriver.Chrome(options = options)

In [189]:
def enter_facebook(browser, login, password):
    """Login to fb

    :param login: (str) fb login
    :param password: (str) fb password

    :return: -1 if error
    """
    browser.get(__login_url)

    try:
        login_form = browser.find_element_by_xpath("//input[@id='email' or @name='email']")
        login_form.clear()
        login_form.send_keys(login)
    except Exception as e:
        print(e)
        print('error: email')
        return -1

    try:
        pass_form = browser.find_element_by_xpath("//input[@id='pass' or @name='pass']")
        pass_form.clear()
        pass_form.send_keys(password)
    except Exception as e:
        print(e)
        print('error: password')
        return -1

    try:
        browser.find_element_by_xpath("//button[@id='loginbutton']").click()
    except Exception as e:
        print(e)
        print('error: login button')
        return -1

    if 'login_attempt' in browser.current_url:
        return -1

In [190]:
enter_facebook(browser, FB_LOGIN, FB_PASSWORD)

In [12]:
# posts = fb.get_posts('100032357613324')
page_id = '100032357613324'
browser.get(__base_url + '/' + page_id)

page_soup = BeautifulSoup(browser.page_source, 'lxml')

In [15]:
nonavbl = page_soup.find('title', text=re.compile('.*Страница не найдена.*'))
if nonavbl:
    print('Страница не найдена')
nonavbl = page_soup.find('h2', text=re.compile('.*Эта страница недоступна.*'))
if nonavbl:
    print('Эта страница недоступна')
nonavbl = page_soup.find('h2', text=re.compile('.*К сожалению, этот контент сейчас недоступен.*'))
if nonavbl:
    print('К сожалению, этот контент сейчас недоступен')

In [16]:
page_soup.find('title', text=re.compile('.*Страница не найдена.*'))

### Блок для замены функции поиска постов

In [250]:
from time import sleep 
from datetime import datetime, timedelta

from selenium.webdriver.common.action_chains import ActionChains
actions = ActionChains(browser)
limit = 20

In [321]:
date1 = '24 марта в 10:20'
date2 = '1 ч.'
date3 = '11 февраля'
date4 = '2 д.'
date5 = '30 мин.'

textToDate_months = {
    'января': 1,
    'февраля': 2,
    'марта': 3,
    'фпреля': 4,
    'мая': 5,
    'июня': 6,
    'июля': 7,
    'августа': 8,
    'сентября': 9,
    'октября': 10,
    'ноября': 11,
    'декабря': 12
}

def textToDate(text):        
    date = datetime.today()
    if text:  
        s = text.split()
        t = int(s[0])
        if 'ч.' in s[1]:
            date -= timedelta(hours=t)
        elif 'д.' in s[1]:
            date -= timedelta(days=t)
        elif 'мин.' in s[1]:
            date -= timedelta(minutes=t)
        else:
            date = datetime(date.year, int(textToDate_months.get(s[1], date.month)), t)                
    return date
    

[textToDate(d) for d in [date1, date2, date3, date4, date5]]

[datetime.datetime(2021, 3, 24, 0, 0),
 datetime.datetime(2021, 4, 10, 18, 33, 5, 628457),
 datetime.datetime(2021, 2, 11, 0, 0),
 datetime.datetime(2021, 4, 8, 19, 33, 5, 628457),
 datetime.datetime(2021, 4, 10, 19, 3, 5, 628457)]

In [317]:
def getPostDate(browser, post):
    browser.execute_script("arguments[0].scrollIntoView();", post)
    abilities = post.find_elements_by_xpath('.//span/a/span')
    if len(abilities)> 0:
        return textToDate(abilities[0].text)
    return datetime.today()            

In [197]:
# try:
#     WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.TAG_NAME, "div[role='article']")))
# except TimeoutException:
#     print(posts_list) 

In [351]:
from_date = datetime.today() - timedelta(days=2)

In [376]:
postcount = -1
posts_list = []
error_count = 0
mindate = datetime.today()

while ((not from_date and len(posts_list) < limit) or (from_date and mindate >= from_date)) and len(posts_list) != postcount:
    postcount = len(posts_list)
    for i in range(postcount // 3 + 1):
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(0.5)    
    posts_list = browser.find_elements_by_tag_name("div[role='article'][aria-posinset]")
    if len(posts_list)>0:
        mindate = getPostDate(browser, posts_list[-1])
    print(len(posts_list), postcount)

if from_date:
    posts_list = [post for post in posts_list if getPostDate(browser, post) >= from_date]
    
print(len(posts_list), postcount)

5 0
11 5
19 11
35 19
35 19


In [381]:
posts_xpath = "div[role='article'][aria-posinset]"
post_more_button = './/div[contains(text(), "Ещё")]'
post_comment_more_button = './/span[starts-with(text(), "Посмотреть ещё")]'
post_comment_more_button2 = './/span[starts-with(text(), "Показать ещё")]'
need_comments = True

result = []
for ind, post in enumerate(posts_list):
    try:
        postDate = getPostDate(browser, post)
        # раскрыть ещё, если есть
        if not from_date or postDate >= from_date:
            more = post.find_elements_by_xpath(post_more_button)
            if more:
                more[0].click()

            # раскрыть все комменты, если надо
            if need_comments:
                more_comments_finder = lambda: post.find_elements_by_xpath(
                    post_comment_more_button) + post.find_elements_by_xpath(
                    post_comment_more_button2)
                more_comments = more_comments_finder()
                while more_comments:
                    more_comments[0].click()
                    sleep(1)
                    more_comments = more_comments_finder()

            result.append(BeautifulSoup(post.get_attribute('innerHTML'), 'lxml'))
    except Exception as e:
        print(f'Пост {ind + 1} пропущен\n', e)

In [367]:
post = posts_list[0]

In [372]:
browser.execute_script("arguments[0].scrollIntoView();", post)
more = post.find_elements_by_xpath('.//div[contains(text(), "Ещё")]')
if more:
    more[0].click()

In [373]:
   
more_comments_finder = lambda: post.find_elements_by_xpath('.//span[starts-with(text(), "Посмотреть ещё")]') + post.find_elements_by_xpath('.//span[starts-with(text(), "Показать ещё")]')
more_comments = more_comments_finder()
while more_comments:
    more_comments[0].click()
    sleep(1)
    more_comments = more_comments_finder()    
    
comments = post.find_elements_by_tag_name("ul")[1]

In [ ]:
print(len(posts_list))
dates = [getPostDate(browser, post) for post in posts_list]
print(len(dates))
dates
# post.get_attribute('innerHTML')

## Конструктор поста

In [394]:
post_soup = result[2]

In [384]:
def get_id(post_soup):
    try:
        post_url = post_soup.find('a', attrs={'href': re.compile(r'.*/posts/.*')})['href']
        return re.search(r'/posts/([0-9]+)', post_url).group(1)
    except Exception as e:
        print(e)
        print('crashed while reading id')
        # return ''
        pass

    try:
        img_url = post_soup.find('a', attrs={'href': re.compile(r'.*/photos/[a-z]*\.[0-9]+/[0-9]+/')})['href']
        return re.search(r'/photos/[a-z]+\.[0-9]+/([0-9]+)/', img_url).group(1)
    except Exception as e:
        print(e)
        pass

    try:
        permalink = post_soup.find('a', attrs={'href': re.compile(r'.*/permalink/[0-9]+/.*')})['href']
        return re.search(r'.*/permalink/([0-9]+)/.*', permalink).group(1)
    except Exception as e:
        print(e)
        return ''

In [386]:
def get_icon_url(post_soup):
    try:
        return post_soup.find('image')['xlink:href']
    except Exception as e:
        print(e)
        print('crashed while reading comment icon')
        return None

In [409]:
def get_message(post_soup):
    try:
        content = post_soup.find('div', attrs={'dir': 'auto'}).find('span', attrs={'dir': 'auto'}).findAll('div', recursive=False)
        return '\n'.join(list(map(lambda x: x.text, content)))
    except Exception as e:
        print(e)
        print('crashed while reading message')
        return ''

In [411]:
def get_date(post_soup):
    """Get post's date

    :param post_soup: (object) post soup

    :return: (int) Unix Timestamp (None if error)
    """
    try:
        date = post_soup.find('abbr')['data-utime']
        return datetime.fromtimestamp(int(date))
    except Exception as e:
        print(e)
        print('crashed while searching date')
        return None

In [412]:
get_date(post_soup)
# get_comments_count(post_soup)
# get_likes(post_soup)
# get_reposts_count(post_soup)
# get_comments(post_soup)

'NoneType' object is not subscriptable
crashed while searching date


In [ ]:
def get_comments_count(post_soup):
    """Get count of post's comments

    :param post_soup: (object) post soup

    :return: (int) count of comments (0 if error)
    """
    try:
        coments_count = post_soup.find('a', text=re.compile(r'^\s*Комментарии:\s*\d+\s*$'))
        return int(re.search(r'\d+', coments_count.text).group(0))
    except Exception as e:
        print(e)
        print('crashed while searching number of comments')
        return 0

In [ ]:
def get_likes(post_soup):
    """Get count of post's likes

    :param post_soup: (object) post soup

    :return: (int) count of likes (0 if error)
    """
    try:
        likes = post_soup.find('span', attrs={'class': '_3dlh'})
        return int(likes.text)
    except Exception as e:
        print(e)
        print('crashed while counting likes')
        return 0

In [ ]:
def get_reposts_count(post_soup):
    """Get count of post's reposts

    :param post_soup: (object) post soup

    :return: (int) count of reposts (0 if error)
    """
    try:
        coments_count = post_soup.find('a', text=re.compile(r'^\s*Поделились:\s*\d+\s*$'))
        return int(re.search(r'\d+', coments_count.text).group(0))
    except Exception as e:
        print(e)
        print('crashed while searching number of reposts')
        return 0

In [ ]:
def get_comments(post_soup):
    """Get comments for post

    :param post_soup: (object) post soup

    :return: (list) comments
    """
    try:
        comment_el_list = []
        for div in post_soup.findAll(lambda x: x.name == 'div' and not x.attrs):
            l = div.findAll('div', attrs={'aria-label': re.compile(".*[Кк]омментарий.*")})
            if len(l) > 0: comment_el_list += l
        return [Comment(com_el) for com_el in comment_el_list if Comment(com_el).message is not None]
    except Exception as e:
        print(e)
        print('crashed while searching comments')
        return []

In [407]:
for post_soup in result:
    content = post_soup.find('div', attrs={'dir': 'auto'}).find('span', attrs={'dir': 'auto'}).findAll('div', recursive=False)
    print('\n'.join(list(map(lambda x: x.text, content))))
    print()

    # post_soup.find('div', attrs={'dir': 'auto'}).findAll('div')

Во Франции аномальные холода, пришедшие в большинство регионов страны, уничтожили значительную часть будущего урожая винограда. В начале апреля по ночам в некоторых провинциях температура опускалась до минус восьми градусов по Цельсию; многие фермеры пытались согреть виноградники, зажигая свечи на участках. Из-за заморозков сборы винограда могут стать наименьшими за последние 40 лет.

Из-за пандемии коронавируса количество участников церемонии будет ограничено. Помимо носильщиков гроба, декана часовни в Виндзоре и архиепископа Кентерберийского, на похоронах будут присутствовать 30 человек. Среди них будет принц Гарри, который прилетит на похороны из США.

Сегодняшний эпизод «Что случилось» о конфликте в Донбассе — первый подкаст, который мы записали на видео! Признаемся, все произошло довольно спонтанно. Мы только в начале недели решили, что будем снять субботний подкаст на видео. И спустя четыре дня — камера, свет, декорации. Да, мы иногда вот так импровизируем.
Смотрите разговор Андр